In [1]:
%matplotlib ipympl
import numpy as np
import cv2
import pickle
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *
import sys
sys.path.append("..")
from ssc import ssc

In [2]:
last_keyframe_dict = pickle.load(open("../last_keyframe.pkl", "rb"))
new_keyframe = pickle.load(open("../new_keyframe.pkl", "rb"))
new_kp = pickle.load(open("../new_kp.pkl", "rb"))
new_pose = pickle.load(open("../new_pose.pkl", "rb"))

In [3]:
last_keyframe = last_keyframe_dict["frame"]
last_kp = last_keyframe_dict["kp_matched"]
#last_kp = last_keyframe_dict["kp"]
last_des = last_keyframe_dict["des"]
last_pose = last_keyframe_dict["pose"]

In [4]:
vis_last_keyframe = cv2.drawKeypoints(np.copy(last_keyframe), cv2.KeyPoint_convert(last_kp.reshape(-1, 1, 2)), None, color=(0,0,255))
vis_new_keyframe = cv2.drawKeypoints(np.copy(new_keyframe), cv2.KeyPoint_convert(new_kp.reshape(-1, 1, 2)), None, color=(0,0,255))

In [5]:
# test if feature points correspond (-> result: they do)
vis_last_keyframe = cv2.circle(vis_last_keyframe, tuple(last_kp[-1].astype(np.int)), 20, (255,0,0), thickness=2)
vis_new_keyframe = cv2.circle(vis_new_keyframe, tuple(new_kp[-1].astype(np.int)), 20, (255,0,0), thickness=2)

In [6]:
# extract features in both keyframes
num_ret_points = 3000
tolerance = 0.1
orb = cv2.ORB_create()
fast = cv2.FastFeatureDetector_create(threshold=12)
last_kp = fast.detect(last_keyframe, None)
last_kp = sorted(last_kp, key = lambda x:x.response, reverse=True)
last_kp = ssc(last_kp, num_ret_points, tolerance, last_keyframe.shape[1], last_keyframe.shape[0])
last_kp, last_des = orb.compute(last_keyframe, last_kp)

new_kp = fast.detect(new_keyframe, None)
new_kp = sorted(new_kp, key = lambda x:x.response, reverse=True)
new_kp = ssc(new_kp, num_ret_points, tolerance, new_keyframe.shape[1], new_keyframe.shape[0])
new_kp, new_des = orb.compute(new_keyframe, new_kp)

In [7]:
# draw new keypoints in blue
vis_new_keyframe = cv2.drawKeypoints(vis_new_keyframe, new_kp, None, color=(255,0,0))
vis_last_keyframe = cv2.drawKeypoints(vis_last_keyframe, last_kp, None, color=(255,0,0))

In [8]:
# draw frames and kps
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))
ax1.set_title("Last Keyframe")
ax1.imshow(vis_last_keyframe[:, :, ::-1])
ax2.set_title("New Keyframe")
ax2.imshow(vis_new_keyframe[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

In [9]:
# match new key points with those of last KF
num_matches = 3000
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(last_des, new_des)
matches = sorted(matches, key = lambda x:x.distance)
last_pts = np.array([last_kp[m.queryIdx].pt for m in matches[:num_matches]]).reshape(1, -1, 2)
current_pts = np.array([new_kp[m.trainIdx].pt for m in matches[:num_matches]]).reshape(1, -1, 2)
last_des = [last_des[m.queryIdx] for m in matches[:num_matches]]
current_des = [new_des[m.trainIdx] for m in matches[:num_matches]]

match_frame = cv2.drawMatches(last_keyframe, last_kp, new_keyframe, new_kp, matches[:num_matches], None, matchColor=(0, 0, 0), flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [10]:
f, ax = plt.subplots(1,1, figsize=(13, 3))
ax.imshow(match_frame[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

### Triangulate new map points from existing keypoint matches

In [11]:
# when triangulating between last KF and new KF, map points will be expressed w.r.t. last KF
# need to transform new map points with pose of last KF to transform new map points to world coords

In [12]:
def from_twist(twist):
    """Convert a 6D twist coordinate (shape (6,)) into a 3x3 rotation matrix
    and translation vector (shape (3,))."""
    r = twist[:3]
    t = twist[3:].reshape(3, 1)
    R, _ = cv2.Rodrigues(r)
    return R, t

R1, t1 = from_twist(last_pose)
R2, t2 = from_twist(new_pose)

In [13]:
R1, t1

(array([[ 9.99997451e-01,  6.19493807e-04, -2.17136692e-03],
        [-6.18710447e-04,  9.99999743e-01,  3.61421246e-04],
        [ 2.17159026e-03, -3.60076878e-04,  9.99997577e-01]]),
 array([[ 0.00762682],
        [-0.99653823],
        [-0.08278523]]))

In [14]:
R2, t2

(array([[ 0.99993232, -0.00628835, -0.00978844],
        [ 0.00626896,  0.99997833, -0.00200981],
        [ 0.00980087,  0.00194831,  0.99995007]]), array([[ 0.09403594],
        [-3.81790881],
        [-0.29770549]]))

In [15]:
w = 1920
h = 1080
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

In [16]:
# create projection matrices needed for triangulation of initial 3D point cloud
proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
proj_matrix1 = camera_matrix.dot(proj_matrix1)
proj_matrix2 = camera_matrix.dot(proj_matrix2)

In [17]:
pts_3d = cv2.triangulatePoints(proj_matrix1, proj_matrix2, last_pts.reshape(-1, 2).T, current_pts.reshape(-1, 2).T).T
pts_3d = cv2.convertPointsFromHomogeneous(pts_3d).reshape(-1, 3)

In [22]:
pts_3d

array([[  7.37216774,   3.05765819,  19.1817675 ],
       [  6.51188836,  -8.25209252,  18.65844906],
       [ -7.48784619,  -3.18281143,  17.25596289],
       ...,
       [-10.83610348,  -0.64183189,  19.01094631],
       [  0.066411  ,  -3.45274661,   2.07185262],
       [ 11.10197088,  -3.45761665,  18.25743517]])

In [18]:
# remove points with negative z coordinate
pts_3d = pts_3d[np.where(pts_3d[:, 2] >= t1[2]), :].reshape(-1, 3)

In [19]:
# load previous map points
map_points = pickle.load(open("../map_points.pkl", "rb"))

In [20]:
m = map_points[0]["mask"]
map_points[0]["pts_3d"][m, :]

array([[-2.94850682,  0.80231726, 19.32300943],
       [10.98307193,  1.33775226, 19.51356268],
       [ 5.63523702,  2.24292049, 19.35714455],
       ...,
       [10.19264803, -7.04378431, 19.98336336],
       [12.15729427,  0.37861908, 19.30982126],
       [ 0.29409611, -1.44233714,  2.06248084]])

In [21]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([-15,15])
ax.set_ylim([-15,15])
ax.set_zlim([-25,25])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

plot_basis(ax, np.eye(3), np.zeros((3,)))
plot_basis(ax, R1, t1.reshape(3,))
plot_basis(ax, R2, t2.reshape(3,)) 

# previous map points
m = map_points[0]["mask"]
ax.scatter(map_points[0]["pts_3d"][m, 0], map_points[0]["pts_3d"][m, 1], map_points[0]["pts_3d"][m, 2], s=3, c="blue")

# new map points
ax.scatter(pts_3d[:, 0], pts_3d[:, 1], pts_3d[:, 2], s=3, c="orange")

#out_of_view = np.where(new_kp[:, 1]>=1080)
#ax.scatter(pts_3d[out_of_view, 0], pts_3d[out_of_view, 1], pts_3d[out_of_view, 2], s=3)

plt.show()

FigureCanvasNbAgg()

In [ ]:
# Below: Finding corresponding features along epilines

In [26]:
# compute fundamental matrix
# (1) via matched feature points
#F, F_mask = cv2.findFundamentalMat()

# (2) from the essential matrix

# (3) from the known relative pose of cameras
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
K = camera_matrix


In [27]:
# pick one feature point in the right image and draw its epiline in the left image
points = new_kp.reshape(-1, 1, 2)
lines = cv2.computeCorrespondEpilines(points, 2, F)

NameError: name 'F' is not defined